Otimização da Função de Ackley

Grupo: Anderson Carneiro da Silva (acs6@cin.ufpe.br) e Lucas Thierry Chaves Muniz (ltcm@cin.ufpe.br)

In [26]:
from numpy import pi
from numpy import cos
from numpy import exp
from numpy import sqrt

Função Fitness

In [27]:
def ackley (vector, c1=20, c2=0.2, c3=2*pi):
    d = len(vector)
    sqrt_sum=0
    cos_sum=0
    for elem in vector:
        sqrt_sum += elem**2
        cos_sum += cos(c3*elem)
    
    sqrt_term = -c1 * exp(-c2*(sqrt(sqrt_sum/d)))
    cos_term = -exp(cos_sum/d)

    return sqrt_term + cos_term + c1 + exp(1)

In [28]:
print(ackley([0, 0, 0, 0]))

4.440892098500626e-16


In [29]:
import numpy as np
from abc import ABC, abstractmethod


class Individual(ABC):
    def __init__(self, value=None, init_params=None):
        if value is not None:
            self.value = value
        else:
            self.value = self._random_init(init_params)

    @abstractmethod
    def pair(self, other, pair_params):
        pass

    @abstractmethod
    def mutate(self, mutate_params):
        pass

    @abstractmethod
    def _random_init(self, init_params):
        pass


class Optimization(Individual):
    def pair(self, other, pair_params):
        return Optimization(pair_params['alpha'] * self.value + (1 - pair_params['alpha']) * other.value)

    def discreet_recomb(self, other):
        for i in range(len(self.value)):
            choice = np.random.randint(2)
            if choice==1:
                self.value[i] = other.value[i]
        return Optimization(self.value)

    def intermediate_recomb(self, other, init_params, d=0.25):
            lower_bound=init_params['lower_bound']
            upper_bound=init_params['upper_bound']
            for i in range(len(self.value)):
                spread = self.value[i]-other.value[i]
                inter_min_bound = (min(self.value[i], other.value[i]))-d*spread
                inter_max_bound = (max(self.value[i], other.value[i]))+d*spread
                if inter_min_bound < lower_bound:
                    inter_min_bound = lower_bound
                if inter_max_bound > upper_bound:
                    inter_max_bound = upper_bound
                self.value[i] = np.random.uniform(inter_min_bound, inter_max_bound)
            return Optimization(self.value)

    def mutate(self, mutate_params):
        self.value += np.random.normal(0, mutate_params['rate'], mutate_params['dim'])
        for i in range(len(self.value)):
            if self.value[i] < mutate_params['lower_bound']:
                self.value[i] = mutate_params['lower_bound']
            elif self.value[i] > mutate_params['upper_bound']:
                self.value[i] = mutate_params['upper_bound']

    def _random_init(self, init_params):
        return np.random.uniform(init_params['lower_bound'], init_params['upper_bound'], init_params['dim'])


class Population:
    def __init__(self, size, fitness, individual_class, init_params):
        self.fitness = fitness
        self.individuals = [individual_class(init_params=init_params) for _ in range(size)]
        self.individuals.sort(key=lambda x: self.fitness(x))

    def replace(self, new_individuals):
        size = len(self.individuals)
        self.individuals.extend(new_individuals)
        self.individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = self.individuals[-size:]

    def replace_comma(self, new_individuals):
        size = len(self.individuals)
        if len(new_individuals) < size:
            new_individuals.extend(self.individuals[-size-len(self.individuals):])
        new_individuals.sort(key=lambda x: self.fitness(x))
        self.individuals = new_individuals[-size:]

    def get_parents(self, n_offsprings):
        mothers = self.individuals[-2 * n_offsprings::2]
        fathers = self.individuals[-2 * n_offsprings + 1::2]

        return mothers, fathers


class Evolution:
    def __init__(self, pool_size, fitness, individual_class, n_offsprings, survival_select, \
         recomb_type, pair_params, mutate_params, init_params):
        self.pair_params = pair_params
        self.mutate_params = mutate_params
        self.pool = Population(pool_size, fitness, individual_class, init_params)
        self.n_offsprings = n_offsprings
        self.survival_select=survival_select
        self.recomb_type=recomb_type
        self.init_params=init_params

    def step(self):
        mothers, fathers = self.pool.get_parents(self.n_offsprings)
        offsprings = []

        for mother, father in zip(mothers, fathers):
            if(self.recomb_type == 'simple_pair'):
                offspring = mother.pair(father, self.pair_params)
            if(self.recomb_type == 'discreet'):
                offspring = mother.discreet_recomb(father)
            if(self.recomb_type == 'intermediate'):
                offspring = mother.intermediate_recomb(father, self.init_params)
            offspring.mutate(self.mutate_params)
            offsprings.append(offspring)

        if(self.survival_select=='comma'):
            self.pool.replace_comma(offsprings)
        elif(self.survival_select=='plus'):
            self.pool.replace(offsprings)

In [30]:
def fitness(vector):
    return 1/ackley(vector.value)


xi_min = -15
xi_max = 15
dimensions = 30
generations = 10000
tries_per_params = 15

variable_params_dict_default = {'pool_size': 30, 'fitness': fitness, 'individual_class': Optimization, \
    'n_offsprings': 200, 'survival_select': 'plus', 'recomb_type': 'intermediate'}
pair_params_dict_default = {'alpha': 0.5}
mutate_params_dict_default = {'lower_bound': xi_min, 'upper_bound': xi_max, 'rate': 0.5, 'dim': dimensions}
init_params_dict_default = {'lower_bound': xi_min, 'upper_bound': xi_max, 'dim': dimensions}

surv_list = ['comma', 'plus']
recomb_list = ['simple_pair', 'discreet', 'intermediate']

for recomb in recomb_list:
    for surv in surv_list:
        tries_fitness_list=[]
        for z in range(tries_per_params):
            variable_params_dict = variable_params_dict_default
            pair_params_dict = pair_params_dict_default
            mutate_params_dict = mutate_params_dict_default
            init_params_dict = init_params_dict_default
            variable_params_dict['survival_select'] = surv
            variable_params_dict['recomb_type'] = recomb
            evo = Evolution(
                pool_size = variable_params_dict['pool_size'], fitness = variable_params_dict['fitness'], \
                    individual_class = variable_params_dict['individual_class'], \
                        n_offsprings = variable_params_dict['n_offsprings'], \
                            survival_select = variable_params_dict['survival_select'], \
                                recomb_type = variable_params_dict['recomb_type'],
                pair_params=pair_params_dict,
                mutate_params=mutate_params_dict,
                init_params=init_params_dict
            )

            for i in range(generations):
                evo.step()
                best_individual = evo.pool.individuals[-1]
                best_fitness = fitness(best_individual)
                if best_fitness > (1/0.0001):
                    break

            tries_fitness_list.append(best_fitness)
            #print(best_individual.value)
            #print(best_fitness)
        print(f'{recomb} Recombination - {surv} Survival - average best fitness {np.mean(tries_fitness_list)}')

simple_pair Recombination - comma Survival - average best fitness 0.05517234728261143
simple_pair Recombination - plus Survival - average best fitness 0.361652859316114
discreet Recombination - comma Survival - average best fitness 0.05536833069784055
discreet Recombination - plus Survival - average best fitness 0.05468005014777788
intermediate Recombination - comma Survival - average best fitness 0.05499938328775523
intermediate Recombination - plus Survival - average best fitness 0.05534510322877143
